In [1]:
# -*- coding: UTF-8 -*-

In [12]:
ls -l ./scoring

total 7832
-rw-r--r--   1 yehua  staff  326206 May  3 12:36 10.tra
-rw-r--r--   1 yehua  staff  322138 May  3 12:36 11.tra
-rw-r--r--   1 yehua  staff  317665 May  3 12:36 12.tra
-rw-r--r--   1 yehua  staff  313328 May  3 12:36 13.tra
-rw-r--r--   1 yehua  staff  309390 May  3 12:36 14.tra
-rw-r--r--   1 yehua  staff  305906 May  3 12:36 15.tra
-rw-r--r--   1 yehua  staff  302505 May  3 12:36 16.tra
-rw-r--r--   1 yehua  staff  299137 May  3 12:36 17.tra
-rw-r--r--   1 yehua  staff  342203 May  3 12:36 7.tra
-rw-r--r--   1 yehua  staff  336249 May  3 12:36 8.tra
-rw-r--r--   1 yehua  staff  331092 May  3 12:36 9.tra
drwxrwxr-x  30 yehua  staff    1020 May  3 12:38 decode/
drwxrwxr-x  24 yehua  staff     816 May  3 12:36 log/
-rw-r--r--   1 yehua  staff  341386 May  3 12:36 test_filt.txt
-rw-r--r--   1 yehua  staff  139160 May  3 12:37 words.txt


In [3]:
%%bash
less ../exp/tri2b/decode/scoring/17.tra | tail -2

NI45FBP_0101_2529230_2536840 9338 4946 4131 10104 11175 11147 6484 9157 9301 9311 8786 9616 8786 7663 11091 10794 8886 10938 10279 8771 
NI45FBP_0101_2536840_2547460 8449 959 3581 7900 7758 3975 10938 8850 3940 7652 8337 3581 2210 6444 2 5489 9209 8850 7663 


In [4]:
%%bash 
less ../exp/tri2b/decode/scoring/test_filt.txt | tail -2

UI29FAZ_0104_0975410_0977779 后 悔 是 OK SIL154
UI29FAZ_0104_0985128_0986293 ERR


In [5]:
%%bash
less ../data/lang/words.txt | head -5

<eps> 0
<oov> 1
A 2
ABACK 3
ABALONE 4


In [5]:
import os
from collections import defaultdict
import numpy as np

In [10]:
parent_path = os.path.split(os.getcwd())[0]
print ("parent path is {}".format(parent_path))

parent path is /Volumes/STARTRACK/deep-learning


In [8]:
def get_file_list(path, extension):
    files = os.listdir(path)
    files = [f for f in files if f.endswith(extension)]
    print (files)
    return files

In [14]:
tra_path

'/Volumes/STARTRACK/deep-learning/scoring'

In [15]:
os.listdir(tra_path)

['.ipynb_checkpoints', 'kaldi-16-key', 'MER_score.ipynb', 'scoring']

In [41]:
#tra_path = parent_path + "/exp/tri2b/decode/scoring/"
tra_path = parent_path + "/scoring/scoring/"
tra_files = get_file_list(tra_path, ".tra") #need to get tri2b from terminal

['10.tra', '11.tra', '12.tra', '13.tra', '14.tra', '15.tra', '16.tra', '17.tra', '7.tra', '8.tra', '9.tra']


In [24]:
def gen_symbol_map():
    r"""
    Returns
    -------
    dic : defaultdict
      a dictionary with key as word symbol and value as the corresponding word
      e.g. '3': 'ABACK' 
    """
    dic = defaultdict(str)
    # words_path = "/data/lang/words.txt"
    words_path = "/scoring/scoring/words.txt"
    with open(parent_path + words_path, "r") as inputfile:
        for line in inputfile:
            word = line.split(" ")[0]
            symbol = line.strip("\n").split(" ")[1]
            dic[symbol] = word
    return dic
    

In [25]:
s2w = gen_symbol_map()

In [26]:
len(s2w)

11360

In [27]:
s2w['9338']

'因为'

In [28]:
def gen_ground_truth_map(model):
    r"""
    Returns
    -------
    dic : defaultdict
      a dictionary with utterance id as key and groud truth transcript as its value 
    """
    dic = defaultdict(list)
    #filt_path = "/exp/" + model + "/decode/scoring/test_filt.txt"
    filt_path = "/scoring/scoring/test_filt.txt"
    with open(parent_path + filt_path, "r") as inputfile:
        for line in inputfile:
            utterance_id = line.split(" ")[0]
            words = line.strip("\n").split(" ")[1:]
            dic[utterance_id] = words
    return dic


In [29]:
truth_map = gen_ground_truth_map("tri2b")

In [30]:
truth_map['NC01FBX_0101_0165090_0167860']

['不',
 '懂',
 'BUT',
 'OFFICIAL',
 'RESULT',
 '还没有',
 '出',
 'I',
 'THINK',
 '出',
 '了',
 '他们',
 '就会']

In [31]:
# compare two lists and find difference 
temp1 = ['那种', 'BUDGET', '现在', '还没有', '出来', '得', '出', '他们', '就会']# predicted
temp2 = ['不','懂','BUT','OFFICIAL','RESULT','还没有','出','I','THINK','出','了','他们','就会'] # truth
list(set(temp1) - set(temp2))

['BUDGET', '那种', '出来', '得', '现在']

In [32]:
# compare two lists and find match
set(temp2).intersection(temp1)

{'他们', '出', '就会', '还没有'}

In [33]:
def isEnglish(word):
    from nltk.corpus import wordnet
    if word == "SIL1" or word == "SIL2":
        return -1
    if not wordnet.synsets(word):
        return 0
    else:
        return 1

In [34]:
isEnglish("look")

1

In [35]:
def process_single_sentence(sen1, sen2):
    r"""sen2 as the ground truth, and sen1 as decoded label
    Returns
    -------
    number of match,
    number of english word match,
    number of chinese word match,
    number of silence phones match,
    total number of the truth sentence,
    total number of true words that are english,
    total number of true words that are chinese,
    total number of true words that are sil
    """
    intersection = set(sen2).intersection(sen1)
    # accuracy = len(intersection)/len(sen2)
    
    eng_inter = len([w for w in list(intersection) if isEnglish(w) == 1]) 
    # how many currect english word is captured 
    chi_inter = len([w for w in list(intersection) if isEnglish(w) == 0]) 
    # how many currect chinese word is capture
    sil_inter = len([w for w in list(intersection) if isEnglish(w) == -1]) 
    # how many engligh are there in ground truth
    total_true_eng = len([w for w in list(sen2) if isEnglish(w) == 1]) 
    # how many chinese are there in ground truth
    total_true_chi = len([w for w in list(sen2) if isEnglish(w) == 0]) 
    # how many SIL are there in ground truth
    total_true_sil = len([w for w in list(sen2) if isEnglish(w) == 0]) 
    
    return [len(intersection), eng_inter, chi_inter, sil_inter, len(sen2),total_true_eng, total_true_chi,total_true_sil ]
    

In [36]:
isEnglish("SIL1")

-1

In [37]:
a = [1,2,3]

In [38]:
a[-2]

2

In [39]:
def wer0(r, h):
    """
    credit to:
    base author: Martin Thoma 
    url:https://martin-thoma.com/word-error-rate-calculation/
    modifitication author: Emily Hua
    
    Calculation of WER with Levenshtein distance.

    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.

    Parameters
    ----------
    r : list
    h : list

    Returns
    -------
    int

    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation
    import numpy
    
    g_substitution = 0 # global 
    g_insertion = 0 
    g_deletion = 0
    
    d = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8)
    d = d.reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        for j in range(len(h)+1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i
    #print(d)

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitution = d[i-1][j-1] + 1
                insertion    = d[i][j-1] + 1
                deletion     = d[i-1][j] + 1
                
                g_substitution += substitution
                g_insertion += insertion
                g_deletion += deletion
                
                d[i][j] = min(substitution, insertion, deletion)
    #print (d)

    #return d[len(r)][len(h)], d[len(r)-1][len(h)-1], d[len(r)][len(h)-1], d[len(r)-1][len(h)]
    return [d[len(r)][len(h)]]


In [42]:
variables = ['WER', 'Ins', 'Sub', 'Del', 'Eng Ins', 'Eng Sub', 'Eng Del',
             'Che Ins', 'Che Sub', 'Che Del', 'Total Eng', 'Total Che']
stacked = [0]*1
stats_stacked = [0]*1
counter = 0
for f in tra_files:
    with open(tra_path + f, "r") as inputfile:
        stacked = [0]*1
        for line in inputfile:
            
            #print ("\n" + line)
            symbols = line.replace("\n", "").split(" ")[1:]
            #print (symbols)
            decoded_words = [s2w[sym] for sym in symbols][:-1] # remove "" caused by newline 
            #print (decoded_words)
            true_words = truth_map[line.split(" ")[0]]
            #print (true_words)
            merret = wer0(true_words, decoded_words) #r,h
            #print (merret)
            #print (merret)
            #processed = [merret[var] for var in variables]
            #print (processed)
            stacked = np.vstack((stacked, merret))
    stacked = stacked[1:]
    print (stacked.shape)
    stats = np.sum(stacked, axis=0)
    print (stats)
    stats_stacked = np.vstack((stats_stacked, stats))  
# stats_stacked contains:    
#[chi_WER, eng_WER, sil_WER, MER]
stats_stacked = stats_stacked[1:]
#print (stats_stacked)
means = np.mean(stats_stacked, axis=0)
# mins = np.min(stats_stacked, axis=0)
# maxs = np.max(stats_stacked, axis=0)


print ("\n average: MER {:.3f} ".format(means[0]))
# print ("\n min: MER {:.3f}, eng_WER: {:.3f} , min_WER: {:.3f} ".format(mins[0], mins[1], mins[-1]))
# print ("\n max: chi_WER {:.3f}, eng_WER: {:.3f} , max_WER: {:.3f} ".format(maxs[0], maxs[1], maxs[-1]))

(3392, 1)
[31215]
(3392, 1)
[31199]


KeyboardInterrupt: 

In [238]:
h = ["what", "a", "bright", "day"]
r = ["what", "a", "day"]
wer(r,h)

{'Che Cor: ': 0,
 'Che Del': 0,
 'Che Ins': 0,
 'Che Sub': 0,
 'Cor': 1,
 'Del': 0,
 'Eng Cor': 1,
 'Eng Del': 0,
 'Eng Ins': 1,
 'Eng Sub': 2,
 'Ins': 1,
 'Sub': 2,
 'Total Che': 0,
 'Total Eng': 3,
 'WER': 1.0}

In [47]:
#### functions ####
# modified from https://segmentfault.com/q/1010000000732038
# to detect english character and hyphen and ' and . ONLY
def isAlphahyphen(word):
    try:
        float(word)
        return True
    except:
        if word == '-':
            return True
        elif word == "'":
            return True
        elif word == "'":
            return True   
        else:
            try:
                return word.replace('-','').replace("'",'').replace('.','').encode('ascii').isalnum()
            except UnicodeEncodeError:
                if word == '-':
                    return True
                else:
                    return False

In [160]:
isAlphahyphen("Emily")

True

In [49]:
len("okay")

4

In [50]:
def split1():
    test = ["我知道了", "睡了","okay", "fine"]

    chi = [list(ele) for ele in test if not isAlphahyphen(ele) and len(ele) > 1]
    engs = [ele for ele in test if isAlphahyphen(ele)]
    flattenchi = [item for sublist in chi for item in sublist]
    engs.extend(flattenchi)
    return (engs)

In [51]:
import time

start = time.time()
for i in range(30000):
    split1()
end = time.time()
print(end-start)

0.8447518348693848


In [52]:
[item for sublist in chi for item in sublist]

NameError: name 'chi' is not defined

In [53]:
def split2(test):
    '''split chinese words into character e.g. ["我睡了","okay"] -> ["我", "睡", "了", "okay"]
    Returns
    -------
    newr : list 
      a list of english word and chinese characters 
    '''
    newr = []
    for ele in test:
        if not isAlphahyphen(ele) and len(ele) > 1:
            newr += list(ele)
        else:
            newr += [ele]
    return newr
        

In [76]:
test = ["我睡了"]

In [77]:
import time

start = time.time()
for i in range(30000):
    split2(test)
end = time.time()
print(end-start)

0.16220402717590332


In [161]:
def single_sentence_mer(ref, hyp ,debug=False): #character level for Chinese
#     r = ref.split()
#     h = hyp.split()
    # split chinese words into characters 
    r = split2(ref)
    h = split2(hyp)
    counter = 0
#     r = ref
#     h = hyp
    totalEng = len([ele for ele in r if isAlphahyphen(ele)])
    totalChe = len(r) - totalEng
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
 
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3
    SUB_PENALTY = 1
    INS_PENALTY = 1
    DEL_PENALTY = 1
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
         
    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
     
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                 
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
                 
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    engOkay = 0
    cheOkay = 0
    engSub = 0
    cheSub = 0
    engDel = 0
    cheDel = 0
    engIns = 0
    cheIns = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if isAlphahyphen(r[i]): # is english
                if debug:
                    lines.append("Eng OK\t" + r[i]+"\t"+h[j])
                engOkay += 1
            else:
                if debug:
                    lines.append("Che OK\t" + r[i]+"\t"+h[j])
                cheOkay += 1 
            #lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            
            if isAlphahyphen(r[i]): # is english
                if debug:
                    lines.append("Eng SUB\t" + r[i]+"\t"+h[j])
                engSub += 1 
            else: 
                if debug:
                    lines.append("Che SUB\t" + r[i]+"\t"+h[j])
                cheSub += 1
            #lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            #print (r)
            #print (h)
            #print("i in insertion error {}, and r length is {}".format (i, len(r)))
            if i < len(r):
                if isAlphahyphen(r[i]): # is english
                    if debug:
                        lines.append("Eng INS\t" + r[i]+"\t"+h[j])
                    engIns += 1
                else: 
                    if debug:
                        lines.append("Che INS\t" + r[i]+"\t"+h[j])
                    cheIns += 1
            else:
                
                counter += 1
                #print (counter)
                if isAlphahyphen(r[i-1]):
                     engIns += 1
                else:
                     cheIns += 1
                
            #lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if i < len(r):
                if isAlphahyphen(r[i]): # is english
                    if debug:
                        lines.append("Eng DEL\t" + r[i]+"\t"+h[j])
                    engDel += 1
                else: 
                    if debug:
                        lines.append("Che DEL\t" + r[i]+"\t"+h[j])
                    cheDel += 1
            else:
                #print (counter)
                counter += 1
                if isAlphahyphen(r[i-1]):
                    engDel += 1
                else:
                    cheDel += 1
            #lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = reversed(lines)
        for line in lines:
            print(line)
        print("#cor " + str(numCor))
        print("#sub " + str(numSub))
        print("#del " + str(numDel))
        print("#ins " + str(numIns))
    #return (numSub + numDel + numIns) / (float) (len(r))
    wer_result =  numSub + numDel + numIns
    #print ("index out of bound {} times".format(counter))
    return {'WER':wer_result, 'Cor':numCor, 'Sub':numSub, 'Ins':numIns, 
            'Del':numDel, "Eng Sub": engSub, "Eng Ins": engIns, "Eng Del": engDel, "Eng Cor": engOkay,
             'Che Sub': cheSub, "Che Ins": cheIns, "Che Del": cheDel, "Che Cor: ": cheOkay,
           "Total Eng": totalEng, "Total Che": totalChe, "out of bounds" : counter}

In [138]:
x = np.array([[1,20],[3,4],[99, 0]])
np.argmax(x[:,0],axis=0)

2

In [145]:
len(variables)

12

In [146]:
[var for var in variables]

['Che Sub',
 'Ins',
 'Che Del',
 'Del',
 'Sub',
 'Eng Sub',
 'Total Che',
 'Che Ins',
 'Eng Ins',
 'WER',
 'Total Eng',
 'Eng Del']

In [162]:
variables_map  = {'WER':0, 'Ins':1, 'Sub':2, 'Del':3, 'Eng Ins':4, 'Eng Sub':5, 'Eng Del':6,
             'Che Ins':7, 'Che Sub':8, 'Che Del':9, 'Total Eng':10, 'Total Che':11}
variables = ['WER', 'Ins', 'Sub', 'Del', 'Eng Ins', 'Eng Sub', 'Eng Del',
             'Che Ins', 'Che Sub', 'Che Del', 'Total Eng', 'Total Che']
stacked = [0]*len(variables)
stats_stacked = [0]*10
counter = 0
for f in tra_files:
    with open(tra_path + f, "r") as inputfile:
        stacked = [0]*len(variables)
        for line in inputfile:
            
            #print ("\n" + line)
            symbols = line.replace("\n", "").split(" ")[1:]
            #print (symbols)
            decoded_words = [s2w[sym] for sym in symbols][:-1] # remove "" caused by newline 
            #print (decoded_words)
            true_words = truth_map[line.split(" ")[0]]
            #print (true_words)
            merret = single_sentence_mer(true_words, decoded_words) #r,h
            #print (merret)
            processed = [merret[var] for var in variables]
            #print (processed)
            stacked = np.vstack((stacked, processed))
    stacked = stacked[1:]
    # print (stacked.shape)
    stats = file_level_MER(stacked)
    # print (stats)
    stats_stacked = np.vstack((stats_stacked, stats))  
# stats_stacked contains:    
#[chi_WER, eng_WER, sil_WER, MER]
stats_stacked = stats_stacked[1:]
means = np.mean(stats_stacked, axis=0)
maxs = np.max(stats_stacked, axis=0)
mins_index = np.argmin(stats_stacked[:,variables_map['WER']], axis=0)
min_MER = stats_stacked[mins_index][variables_map['WER']]
min_ins = stats_stacked[mins_index][variables_map['Ins']]
min_sub = stats_stacked[mins_index][variables_map['Sub']]
min_del = stats_stacked[mins_index][variables_map['Del']]
min_eng_ins = stats_stacked[mins_index][variables_map['Eng Ins']]
min_eng_sub = stats_stacked[mins_index][variables_map['Eng Sub']]
min_eng_del = stats_stacked[mins_index][variables_map['Eng Del']]
min_che_ins = stats_stacked[mins_index][variables_map['Che Ins']]
min_che_sub = stats_stacked[mins_index][variables_map['Che Sub']]
min_che_del = stats_stacked[mins_index][variables_map['Che Del']]

print ("\n average: MER {:.3f} ".format(means[0]))
print ("\n min: MER {:.3f}, OVERALL INS {:.3f}, OVERALL SUB {:.3f}, OVERALL DEL {:.3f},\
        ENG INS {:.3f}, ENG SUB {:.3f},ENG DEL {:.3f}, \
        CHE INS {:.3f}, CHE SUB {:.3f}, CHE DEL {:.3f})".format(
        min_MER, min_ins, min_sub, min_del, min_eng_ins, min_eng_sub, min_eng_del, min_che_ins, min_che_sub,  min_che_del, mins[9])
      )

print ('{0:<8} {1:<8} {2:<8} {3:>8}'.format("OVERALL", "ENG", "CHE", "TYPE"))
    
for args in ( (min_ins, min_eng_ins, min_che_ins, "INS" ), 
             (min_sub, min_eng_sub, min_che_sub, "SUB"), (min_del, min_eng_del, min_che_del, "DEL")):
    print ('{0:.3f}   {1:.3f}   {2:.3f}  {3:>8}'.format(*args))
# print ("\n min: MER {:.3f}, eng_WER: {:.3f} , min_WER: {:.3f} ".format(mins[0], mins[1], mins[-1]))
# print ("\n max: chi_WER {:.3f}, eng_WER: {:.3f} , max_WER: {:.3f} ".format(maxs[0], maxs[1], maxs[-1]))
                

sums:  [34041  2261 24100  7680  1241  7997  1627  1020 16103  6053 14277 43241]
57518
sums:  [34049  2065 23771  8213  1141  7888  1775   924 15883  6438 14277 43241]
57518
sums:  [34167  1853 23540  8774  1016  7849  1921   837 15691  6853 14277 43241]
57518
sums:  [34472  1715 23378  9379   953  7799  2095   762 15579  7284 14277 43241]
57518
sums:  [34727  1587 23175  9965   896  7696  2282   691 15479  7683 14277 43241]
57518
sums:  [34996  1440 23042 10514   816  7654  2438   624 15388  8076 14277 43241]
57518
sums:  [35274  1340 22880 11054   764  7587  2595   576 15293  8459 14277 43241]
57518
sums:  [35579  1228 22792 11559   712  7555  2716   516 15237  8843 14277 43241]
57518
sums:  [34901  3175 25630  6096  1690  8441  1246  1485 17189  4850 14277 43241]
57518
sums:  [34387  2819 24918  6650  1525  8230  1402  1294 16688  5248 14277 43241]
57518
sums:  [34097  2533 24441  7123  1388  8075  1524  1145 16366  5599 14277 43241]
57518

 average: MER 0.602 

 min: MER 0.592, OVE

In [97]:
#isEnglish("emily")
isAlphahyphen("EMILY")

True

In [98]:
wer0(r,h)

[2]

In [99]:
h = ["what", "个", "好的", "day"]
r = ["好", "个", "day"]
%time single_sentence_mer(r,h, debug=True)

OP	REF	HYP
['好', '个', 'day']
['what', '个', '好的', 'day']
i in insertion error 2, and r length is 3
Che SUB	好	what
Che OK	个	个
Eng INS	day	好的
Eng OK	day	day
#cor 2
#sub 1
#del 0
#ins 1
CPU times: user 290 µs, sys: 34 µs, total: 324 µs
Wall time: 344 µs


{'Che Cor: ': 1,
 'Che Del': 0,
 'Che Ins': 0,
 'Che Sub': 1,
 'Cor': 2,
 'Del': 0,
 'Eng Cor': 1,
 'Eng Del': 0,
 'Eng Ins': 1,
 'Eng Sub': 0,
 'Ins': 1,
 'Sub': 1,
 'Total Che': 2,
 'Total Eng': 1,
 'WER': 2}

In [101]:
a = [1,2,3]
a[-1]

3

In [84]:
    var_map = {'WER': 0, 'Ins' : 1, 'Sub' : 2, 'Del' : 3, 'Eng Ins' : 4, 'Eng Sub' : 5, 'Eng Del' : 6,
             'Che Ins' : 7, 'Che Sub' : 8, 'Che Del' : 9, 'Total Eng' : 10, 'Total Che' : 11}

In [85]:
  var_map['WER']

0

In [141]:
def file_level_MER(stacked):
    r"""
    Returns
    -------
    ret : list
       MER, sumed Ins, sumed Sub, sumed Del, sumed eng ins, \ 
       sumed eng sub, sumed eng del, sumed che ins, sumed che sub, sumed che del on file level 
    
    """
    var_map = {'WER': 0, 'Ins' : 1, 'Sub' : 2, 'Del' : 3, 'Eng Ins' : 4, 'Eng Sub' : 5, 'Eng Del' : 6,
             'Che Ins' : 7, 'Che Sub' : 8, 'Che Del' : 9, 'Total Eng' : 10, 'Total Che' : 11}
    stacked = stacked[1:]
    sums = np.sum(stacked, axis=0)
    print("sums: ", sums)
    N = sums[var_map['Total Eng']] + sums[var_map['Total Che']]
    print ( N)
    eng_ins = sums[var_map['Eng Ins']] / N
    eng_sub = sums[var_map['Eng Sub']] / N
    eng_del = sums[var_map['Eng Del']] / N
    che_ins = sums[var_map['Che Ins']] / N
    che_sub = sums[var_map['Che Sub']] / N
    che_del = sums[var_map['Che Del']] / N    
    MER = sums[var_map['WER']] / N
    ret = [MER, sums[var_map['Ins']]/N, sums[var_map['Sub']]/N, sums[var_map['Del']]/N, 
           eng_ins, eng_sub, eng_del, che_ins, che_sub, che_del]
    return ret
    

In [87]:
0.0074081857825881361+ 0.21887248463195502+ 0.045794147007828509+ 0.0080071454841590915+ 0.48528345505175224+ 0.12321756948457942

0.8885829874428623

In [88]:
0.88858298744286235

0.8885829874428623

In [89]:
a = [0.89253141680326142, 0.0060942293952002688, 0.21504223931408398, 0.05188500819568781, 0.0060732148110788885, 0.47574916992392718, 0.13768755516328332]

In [90]:
sum(a[1:]) == a[0]

True

In [92]:
def gen_file_stats(stacked):
    stacked = stacked[1:]
    sums = np.sum(stacked, axis=0)
    
    eng_WER = 1-(sums[1]/sums[-3])
    chi_WER = 1-(sums[2]/sums[-2])
    sil_WER = 1-(sums[3]/sums[-1])
    MER = 1-(sums[0]/sums[-4])
    return [chi_WER, eng_WER, sil_WER, MER]

In [214]:
[isEnglish(i) for i in temp1]

[0, 1, 0, 0, 0, 0, 0, 0, 0]

In [215]:
stacked = [0]*8
stats_stacked = [0]*4
counter = 0
for f in tra_files:
    with open(tra_path + f, "r") as inputfile:
        for line in inputfile:
                #print (line)
                symbols = line.replace("\n", "").split(" ")[1:]
                #print (symbols)
                decoded_words = [s2w[sym] for sym in symbols][:-1] # remove "" caused by newline 
                #print (decoded_words )
                true_words = truth_map[line.split(" ")[0]]
                #print (true_words)
                processed = process_single_sentence(decoded_words, true_words)
                stacked = np.vstack((stacked, processed))
    stacked = stacked[1:]
    stats = gen_file_stats(stacked)
    print (stats)
    stats_stacked = np.vstack((stats_stacked, stats))
    
# stats_stacked contains:    
#[chi_WER, eng_WER, sil_WER, MER]
stats_stacked = stats_stacked[1:]
means = np.mean(stats_stacked, axis=0)
chi_WER_mean = means[0]
eng_WER_mean = means[1]
sil_WER_mean = means[-2]
WER_mean = means[-1]
mins = np.min(stats_stacked, axis=0)
maxs = np.max(stats_stacked, axis=0)

print ("\n average: chi_WER {:.3f}, eng_WER: {:.3f} , mean_WER: {:.3f} ".format(chi_WER_mean, eng_WER_mean, WER_mean))
print ("\n min: chi_WER {:.3f}, eng_WER: {:.3f} , min_WER: {:.3f} ".format(mins[0], mins[1], mins[-1]))
print ("\n max: chi_WER {:.3f}, eng_WER: {:.3f} , max_WER: {:.3f} ".format(maxs[0], maxs[1], maxs[-1]))
                
            
                          
            
            
        

[0.66320695916099837, 0.66906878802058967, 1.0, 0.66452317908628589]


KeyboardInterrupt: 

In [29]:
sil_WER_mean

1.0